# Exploratory Data Analysis (EDA) & Improvements
# Facial Emotion Recognition Project

**Purpose**: Comprehensive data exploration, visualization, and model improvement recommendations

**Author**: Enhancement Guide

---

## 📦 Setup & Imports

In [ ]:
# Core libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from collections import Counter
import cv2
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Set style for better visualizations
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

## 🗂️ Data Upload (Google Colab)

In [ ]:
# Uncomment if running in Google Colab
# from google.colab import files
# import zipfile

# uploaded = files.upload()

# for filename in uploaded.keys():
#     with zipfile.ZipFile(filename, 'r') as zip_ref:
#         zip_ref.extractall('.')
#     print(f"Extracted: {filename}")

---
# SECTION 1: Dataset Structure Analysis

In [ ]:
def analyze_dataset_structure(data_dir='Train'):
    """
    Analyze the basic structure and statistics of the dataset
    """
    print("="*80)
    print("DATASET STRUCTURE ANALYSIS")
    print("="*80)
    
    emotions = os.listdir(data_dir)
    emotions = [e for e in emotions if os.path.isdir(os.path.join(data_dir, e))]
    
    stats_data = []
    total_images = 0
    
    for emotion in emotions:
        emotion_path = os.path.join(data_dir, emotion)
        image_files = [f for f in os.listdir(emotion_path) 
                      if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        count = len(image_files)
        total_images += count
        stats_data.append({'Emotion': emotion, 'Count': count})
    
    df_stats = pd.DataFrame(stats_data)
    df_stats['Percentage'] = (df_stats['Count'] / total_images * 100).round(2)
    df_stats = df_stats.sort_values('Count', ascending=False)
    
    print(f"\nTotal Images: {total_images}")
    print(f"Number of Classes: {len(emotions)}\n")
    print(df_stats.to_string(index=False))
    
    # Calculate class imbalance ratio
    max_count = df_stats['Count'].max()
    min_count = df_stats['Count'].min()
    imbalance_ratio = max_count / min_count
    print(f"\nClass Imbalance Ratio: {imbalance_ratio:.2f}:1")
    
    if imbalance_ratio > 1.5:
        print("⚠️  WARNING: Significant class imbalance detected!")
        print("   Recommendation: Use weighted loss or data balancing techniques")
    
    return df_stats

# Run analysis
df_stats = analyze_dataset_structure('Train')

## 📊 Class Distribution Visualization

In [ ]:
def visualize_class_distribution(df_stats):
    """
    Create comprehensive class distribution visualizations
    """
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    # Bar plot
    axes[0].bar(df_stats['Emotion'], df_stats['Count'], 
                color=plt.cm.Set3(range(len(df_stats))))
    axes[0].set_xlabel('Emotion', fontsize=12)
    axes[0].set_ylabel('Number of Images', fontsize=12)
    axes[0].set_title('Class Distribution - Bar Chart', fontsize=14, fontweight='bold')
    axes[0].tick_params(axis='x', rotation=45)
    
    # Add count labels on bars
    for i, (emotion, count) in enumerate(zip(df_stats['Emotion'], df_stats['Count'])):
        axes[0].text(i, count + max(df_stats['Count'])*0.02, str(count), 
                    ha='center', fontweight='bold')
    
    # Pie chart
    colors = plt.cm.Set3(range(len(df_stats)))
    axes[1].pie(df_stats['Count'], labels=df_stats['Emotion'], autopct='%1.1f%%',
                colors=colors, startangle=90)
    axes[1].set_title('Class Distribution - Pie Chart', fontsize=14, fontweight='bold')
    
    # Percentage comparison
    axes[2].barh(df_stats['Emotion'], df_stats['Percentage'], 
                 color=plt.cm.Set3(range(len(df_stats))))
    axes[2].set_xlabel('Percentage (%)', fontsize=12)
    axes[2].set_ylabel('Emotion', fontsize=12)
    axes[2].set_title('Class Distribution - Percentage', fontsize=14, fontweight='bold')
    
    # Add percentage labels
    for i, (emotion, pct) in enumerate(zip(df_stats['Emotion'], df_stats['Percentage'])):
        axes[2].text(pct + 0.5, i, f'{pct}%', va='center', fontweight='bold')
    
    plt.tight_layout()
    plt.show()

# Visualize
visualize_class_distribution(df_stats)

---
# SECTION 2: Image Quality Analysis

In [ ]:
def analyze_image_properties(data_dir='Train', sample_size=100):
    """
    Analyze image properties: dimensions, brightness, contrast, sharpness
    """
    print("\n" + "="*80)
    print("IMAGE QUALITY ANALYSIS")
    print("="*80)
    
    emotions = [e for e in os.listdir(data_dir) 
                if os.path.isdir(os.path.join(data_dir, e))]
    
    properties = {
        'emotion': [],
        'width': [],
        'height': [],
        'brightness': [],
        'contrast': [],
        'aspect_ratio': []
    }
    
    for emotion in emotions:
        emotion_path = os.path.join(data_dir, emotion)
        image_files = os.listdir(emotion_path)[:sample_size]  # Sample for efficiency
        
        for img_file in image_files:
            img_path = os.path.join(emotion_path, img_file)
            try:
                img = Image.open(img_path).convert('L')  # Grayscale
                img_array = np.array(img)
                
                properties['emotion'].append(emotion)
                properties['width'].append(img.width)
                properties['height'].append(img.height)
                properties['brightness'].append(img_array.mean())
                properties['contrast'].append(img_array.std())
                properties['aspect_ratio'].append(img.width / img.height)
                
            except Exception as e:
                print(f"Error processing {img_path}: {e}")
    
    df_props = pd.DataFrame(properties)
    
    # Summary statistics
    print("\nImage Dimension Statistics:")
    print(df_props[['width', 'height']].describe())
    
    print("\nBrightness & Contrast Statistics:")
    print(df_props[['brightness', 'contrast']].describe())
    
    # Check for dimension inconsistencies
    unique_dims = df_props.groupby(['width', 'height']).size()
    if len(unique_dims) > 1:
        print("\n⚠️  WARNING: Multiple image dimensions detected!")
        print("   Recommendation: Ensure all images are resized consistently")
    
    return df_props

# Run analysis
df_props = analyze_image_properties('Train', sample_size=100)

## 📈 Image Property Visualizations

In [ ]:
def visualize_image_properties(df_props):
    """
    Visualize brightness, contrast, and aspect ratio distributions
    """
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # Brightness distribution by emotion
    for emotion in df_props['emotion'].unique():
        subset = df_props[df_props['emotion'] == emotion]['brightness']
        axes[0, 0].hist(subset, alpha=0.6, label=emotion, bins=30)
    axes[0, 0].set_xlabel('Brightness (Mean Pixel Intensity)', fontsize=11)
    axes[0, 0].set_ylabel('Frequency', fontsize=11)
    axes[0, 0].set_title('Brightness Distribution by Emotion', fontsize=13, fontweight='bold')
    axes[0, 0].legend()
    axes[0, 0].grid(alpha=0.3)
    
    # Contrast distribution by emotion
    for emotion in df_props['emotion'].unique():
        subset = df_props[df_props['emotion'] == emotion]['contrast']
        axes[0, 1].hist(subset, alpha=0.6, label=emotion, bins=30)
    axes[0, 1].set_xlabel('Contrast (Std Dev)', fontsize=11)
    axes[0, 1].set_ylabel('Frequency', fontsize=11)
    axes[0, 1].set_title('Contrast Distribution by Emotion', fontsize=13, fontweight='bold')
    axes[0, 1].legend()
    axes[0, 1].grid(alpha=0.3)
    
    # Box plot: Brightness by emotion
    df_props.boxplot(column='brightness', by='emotion', ax=axes[1, 0])
    axes[1, 0].set_xlabel('Emotion', fontsize=11)
    axes[1, 0].set_ylabel('Brightness', fontsize=11)
    axes[1, 0].set_title('Brightness Comparison Across Emotions', fontsize=13, fontweight='bold')
    plt.sca(axes[1, 0])
    plt.xticks(rotation=45)
    
    # Box plot: Contrast by emotion
    df_props.boxplot(column='contrast', by='emotion', ax=axes[1, 1])
    axes[1, 1].set_xlabel('Emotion', fontsize=11)
    axes[1, 1].set_ylabel('Contrast', fontsize=11)
    axes[1, 1].set_title('Contrast Comparison Across Emotions', fontsize=13, fontweight='bold')
    plt.sca(axes[1, 1])
    plt.xticks(rotation=45)
    
    plt.suptitle('')  # Remove auto-generated title
    plt.tight_layout()
    plt.show()

# Visualize
visualize_image_properties(df_props)

---
# SECTION 3: Sample Images Visualization

In [ ]:
def visualize_sample_images(data_dir='Train', n_samples=5):
    """
    Display grid of sample images from each class
    """
    emotions = [e for e in os.listdir(data_dir) 
                if os.path.isdir(os.path.join(data_dir, e))]
    emotions = sorted(emotions)
    
    fig, axes = plt.subplots(len(emotions), n_samples, figsize=(12, 3*len(emotions)))
    
    for i, emotion in enumerate(emotions):
        emotion_path = os.path.join(data_dir, emotion)
        image_files = os.listdir(emotion_path)[:n_samples]
        
        for j, img_file in enumerate(image_files):
            img_path = os.path.join(emotion_path, img_file)
            img = Image.open(img_path).convert('L')
            
            ax = axes[i, j] if len(emotions) > 1 else axes[j]
            ax.imshow(img, cmap='gray')
            ax.axis('off')
            
            if j == 0:
                ax.set_title(f'{emotion.upper()}', fontsize=12, 
                           fontweight='bold', loc='left')
    
    plt.suptitle('Sample Images from Each Emotion Class', 
                fontsize=16, fontweight='bold', y=0.995)
    plt.tight_layout()
    plt.show()

# Visualize samples
visualize_sample_images('Train', n_samples=5)

---
# SECTION 4: Pixel Intensity Analysis

In [ ]:
def analyze_pixel_distributions(data_dir='Train', sample_size=50):
    """
    Analyze pixel intensity distributions across emotion classes
    """
    print("\n" + "="*80)
    print("PIXEL INTENSITY DISTRIBUTION ANALYSIS")
    print("="*80)
    
    emotions = [e for e in os.listdir(data_dir) 
                if os.path.isdir(os.path.join(data_dir, e))]
    
    pixel_stats = {}
    
    for emotion in emotions:
        emotion_path = os.path.join(data_dir, emotion)
        image_files = os.listdir(emotion_path)[:sample_size]
        
        all_pixels = []
        for img_file in image_files:
            img_path = os.path.join(emotion_path, img_file)
            try:
                img = Image.open(img_path).convert('L')
                img_array = np.array(img).flatten()
                all_pixels.extend(img_array)
            except:
                continue
        
        pixel_stats[emotion] = {
            'mean': np.mean(all_pixels),
            'median': np.median(all_pixels),
            'std': np.std(all_pixels),
            'min': np.min(all_pixels),
            'max': np.max(all_pixels)
        }
    
    df_pixel_stats = pd.DataFrame(pixel_stats).T
    print("\nPixel Intensity Statistics by Emotion:")
    print(df_pixel_stats.round(2))
    
    return pixel_stats

# Run analysis
pixel_stats = analyze_pixel_distributions('Train', sample_size=50)

---
# SECTION 5: Data Augmentation Visualization

In [ ]:
def visualize_augmentation_effects(data_dir='Train'):
    """
    Show the effects of different augmentation techniques
    """
    # Get a sample image
    emotions = [e for e in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, e))]
    sample_emotion = emotions[0]
    sample_path = os.path.join(data_dir, sample_emotion)
    sample_file = os.listdir(sample_path)[0]
    image_path = os.path.join(sample_path, sample_file)
    
    # Define augmentations
    augmentations = {
        'Original': transforms.Compose([
            transforms.Grayscale(num_output_channels=1),
            transforms.Resize((48, 48))
        ]),
        'Horizontal Flip': transforms.Compose([
            transforms.Grayscale(num_output_channels=1),
            transforms.Resize((48, 48)),
            transforms.RandomHorizontalFlip(p=1.0)
        ]),
        'Rotation (15°)': transforms.Compose([
            transforms.Grayscale(num_output_channels=1),
            transforms.Resize((48, 48)),
            transforms.RandomRotation(15)
        ]),
        'Translation': transforms.Compose([
            transforms.Grayscale(num_output_channels=1),
            transforms.Resize((48, 48)),
            transforms.RandomAffine(degrees=0, translate=(0.15, 0.15))
        ]),
        'Brightness': transforms.Compose([
            transforms.Grayscale(num_output_channels=1),
            transforms.Resize((48, 48)),
            transforms.ColorJitter(brightness=0.3)
        ]),
        'Combined': transforms.Compose([
            transforms.Grayscale(num_output_channels=1),
            transforms.Resize((48, 48)),
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.RandomRotation(10),
            transforms.RandomAffine(degrees=0, translate=(0.1, 0.1))
        ])
    }
    
    img = Image.open(image_path)
    
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    axes = axes.flatten()
    
    for idx, (name, transform) in enumerate(augmentations.items()):
        transformed = transform(img)
        axes[idx].imshow(transformed, cmap='gray')
        axes[idx].set_title(name, fontsize=12, fontweight='bold')
        axes[idx].axis('off')
    
    plt.suptitle('Data Augmentation Effects', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()

# Visualize augmentations
visualize_augmentation_effects('Train')

---
# SECTION 6: Model Performance Analysis
## (Run this after training your model)

In [ ]:
def analyze_confusion_matrix(model, test_loader, class_names, device):
    """
    Generate and visualize confusion matrix
    """
    from sklearn.metrics import confusion_matrix, classification_report
    
    model.eval()
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            outputs = model(images)
            _, predicted = outputs.max(1)
            
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.numpy())
    
    # Confusion matrix
    cm = confusion_matrix(all_labels, all_preds)
    
    # Visualization
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=class_names, yticklabels=class_names,
                cbar_kws={'label': 'Count'})
    plt.xlabel('Predicted Label', fontsize=12)
    plt.ylabel('True Label', fontsize=12)
    plt.title('Confusion Matrix', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    # Classification report
    print("\nClassification Report:")
    print(classification_report(all_labels, all_preds, target_names=class_names))
    
    return cm

# Example usage (uncomment after training):
# cm = analyze_confusion_matrix(model, test_loader, class_names, device)

---
# SECTION 7: IMPROVEMENT IMPLEMENTATIONS
## 7.1 Weighted Loss for Class Imbalance

In [ ]:
# Calculate class weights from dataset
def compute_class_weights(dataset):
    """
    Compute class weights for imbalanced dataset
    """
    from collections import Counter
    
    # Get all labels
    labels = [label for _, label in dataset]
    class_counts = Counter(labels)
    
    # Calculate weights (inverse frequency)
    total = sum(class_counts.values())
    class_weights = {}
    for cls, count in class_counts.items():
        class_weights[cls] = total / (len(class_counts) * count)
    
    # Convert to tensor
    weights = torch.tensor([class_weights[i] for i in range(len(class_counts))], 
                          dtype=torch.float32)
    
    print("Class Weights:")
    for i, w in enumerate(weights):
        print(f"  Class {i}: {w:.4f}")
    
    return weights

# Example usage:
# train_dataset = datasets.ImageFolder(root='Train', transform=train_transform)
# class_weights = compute_class_weights(train_dataset)
# criterion = nn.CrossEntropyLoss(weight=class_weights.to(device))

## 7.2 Enhanced Data Augmentation

In [ ]:
# Enhanced augmentation pipeline
enhanced_train_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((48, 48)),
    
    # Existing augmentations
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(10),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
    
    # NEW: Additional augmentations
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.RandomPerspective(distortion_scale=0.2, p=0.3),
    transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 2.0)),
    
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5]),
    
    # NEW: Random erasing (simulates occlusions)
    transforms.RandomErasing(p=0.2, scale=(0.02, 0.15), ratio=(0.3, 3.3))
])

print("Enhanced augmentation pipeline created!")
print("New augmentations added:")
print("  ✓ ColorJitter (brightness & contrast)")
print("  ✓ RandomPerspective (viewpoint changes)")
print("  ✓ GaussianBlur (noise robustness)")
print("  ✓ RandomErasing (occlusion simulation)")

## 7.3 Transfer Learning with ResNet

In [ ]:
import torchvision.models as models

def create_resnet_model(num_classes=4, pretrained=True):
    """
    Create ResNet18 model adapted for grayscale emotion recognition
    """
    # Load pretrained ResNet18
    model = models.resnet18(pretrained=pretrained)
    
    # Modify first conv layer for grayscale input
    model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
    
    # Modify final layer for our number of classes
    num_features = model.fc.in_features
    model.fc = nn.Linear(num_features, num_classes)
    
    return model

# Example usage:
# resnet_model = create_resnet_model(num_classes=4).to(device)
# print(f"Total parameters: {sum(p.numel() for p in resnet_model.parameters()):,}")

## 7.4 Attention Mechanism (CBAM)

In [ ]:
class ChannelAttention(nn.Module):
    """Channel Attention Module"""
    def __init__(self, channels, reduction=16):
        super().__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)
        
        self.fc = nn.Sequential(
            nn.Linear(channels, channels // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(channels // reduction, channels, bias=False)
        )
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        b, c, _, _ = x.size()
        
        # Average pooling
        avg_out = self.fc(self.avg_pool(x).view(b, c))
        # Max pooling
        max_out = self.fc(self.max_pool(x).view(b, c))
        
        out = self.sigmoid(avg_out + max_out).view(b, c, 1, 1)
        return x * out

class SpatialAttention(nn.Module):
    """Spatial Attention Module"""
    def __init__(self, kernel_size=7):
        super().__init__()
        self.conv = nn.Conv2d(2, 1, kernel_size, padding=kernel_size//2, bias=False)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        out = torch.cat([avg_out, max_out], dim=1)
        out = self.sigmoid(self.conv(out))
        return x * out

class CBAM(nn.Module):
    """Convolutional Block Attention Module"""
    def __init__(self, channels, reduction=16, kernel_size=7):
        super().__init__()
        self.channel_attention = ChannelAttention(channels, reduction)
        self.spatial_attention = SpatialAttention(kernel_size)
    
    def forward(self, x):
        x = self.channel_attention(x)
        x = self.spatial_attention(x)
        return x

print("CBAM attention module created!")
print("Usage: Add after convolutional blocks to focus on important features")

## 7.5 Advanced Optimizers

In [ ]:
import torch.optim as optim

# AdamW optimizer (better weight decay)
def create_adamw_optimizer(model, lr=0.001, weight_decay=0.01):
    return optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)

# OneCycleLR scheduler
def create_onecycle_scheduler(optimizer, max_lr, steps_per_epoch, epochs):
    return optim.lr_scheduler.OneCycleLR(
        optimizer, 
        max_lr=max_lr, 
        steps_per_epoch=steps_per_epoch, 
        epochs=epochs,
        pct_start=0.3,  # Warm up for 30% of training
        anneal_strategy='cos'
    )

# Example usage:
# optimizer = create_adamw_optimizer(model, lr=0.001, weight_decay=0.01)
# scheduler = create_onecycle_scheduler(optimizer, max_lr=0.01, 
#                                       steps_per_epoch=len(train_loader), 
#                                       epochs=50)

print("Advanced optimizer and scheduler created!")
print("  ✓ AdamW: Better weight decay implementation")
print("  ✓ OneCycleLR: Faster convergence with cyclic learning rate")

## 7.6 Early Stopping

In [ ]:
class EarlyStopping:
    """
    Early stopping to prevent overfitting
    """
    def __init__(self, patience=7, min_delta=0, verbose=True):
        self.patience = patience
        self.min_delta = min_delta
        self.verbose = verbose
        self.counter = 0
        self.best_loss = None
        self.early_stop = False
        self.val_loss_min = np.Inf
    
    def __call__(self, val_loss, model, path='checkpoint.pth'):
        if self.best_loss is None:
            self.best_loss = val_loss
            self.save_checkpoint(val_loss, model, path)
        elif val_loss > self.best_loss - self.min_delta:
            self.counter += 1
            if self.verbose:
                print(f'EarlyStopping counter: {self.counter}/{self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.save_checkpoint(val_loss, model, path)
            self.counter = 0
    
    def save_checkpoint(self, val_loss, model, path):
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} → {val_loss:.6f}). Saving model...')
        torch.save(model.state_dict(), path)
        self.val_loss_min = val_loss

# Example usage:
# early_stopping = EarlyStopping(patience=7, verbose=True)
# 
# for epoch in range(NUM_EPOCHS):
#     train_loss = train_epoch(...)
#     val_loss = validate(...)
#     
#     early_stopping(val_loss, model)
#     if early_stopping.early_stop:
#         print("Early stopping triggered!")
#         break

print("Early stopping implementation ready!")
print("Prevents overfitting by stopping when validation loss stops improving")

## 7.7 Label Smoothing

In [ ]:
class LabelSmoothingCrossEntropy(nn.Module):
    """
    Label smoothing to prevent overconfidence
    """
    def __init__(self, epsilon=0.1, reduction='mean'):
        super().__init__()
        self.epsilon = epsilon
        self.reduction = reduction
    
    def forward(self, preds, targets):
        n_classes = preds.size(-1)
        log_preds = F.log_softmax(preds, dim=-1)
        
        # Smooth labels
        loss = -log_preds.sum(dim=-1)
        if self.reduction == 'mean':
            loss = loss.mean()
        elif self.reduction == 'sum':
            loss = loss.sum()
        
        nll = F.nll_loss(log_preds, targets, reduction=self.reduction)
        
        return (1 - self.epsilon) * nll + self.epsilon * loss / n_classes

# Example usage:
# criterion = LabelSmoothingCrossEntropy(epsilon=0.1)

print("Label smoothing loss function created!")
print("Prevents model from being overconfident in predictions")

## 7.8 Gradient Clipping

In [ ]:
def train_with_gradient_clipping(model, loader, criterion, optimizer, device, max_norm=1.0):
    """
    Training function with gradient clipping
    """
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        
        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=max_norm)
        
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    
    epoch_loss = running_loss / len(loader)
    epoch_acc = 100. * correct / total
    
    return epoch_loss, epoch_acc

print("Gradient clipping training function created!")
print("Prevents exploding gradients during training")

---
# SECTION 8: Summary of Recommendations

## 🎯 Priority Recommendations (Quick Wins)

### Immediate Actions (< 10 minutes each):
1. **✓ Implement weighted loss** for class imbalance
2. **✓ Add enhanced augmentations** (ColorJitter, RandomErasing)
3. **✓ Use AdamW optimizer** instead of Adam
4. **✓ Add gradient clipping** (prevents exploding gradients)
5. **✓ Implement early stopping** (prevents overfitting)

### Short-term Improvements (30-60 minutes):
6. **✓ Try transfer learning** with ResNet18
7. **✓ Implement label smoothing**
8. **✓ Add OneCycleLR scheduler**
9. **✓ Generate confusion matrix** for error analysis
10. **✓ Add CBAM attention** to existing model

### Long-term Enhancements:
- Collect more diverse data
- Implement ensemble methods
- Deploy with FastAPI/Flask
- Create mobile application
- Add Grad-CAM visualization

---

## 📊 Expected Performance Improvements

| Improvement | Expected Accuracy Gain | Difficulty |
|------------|------------------------|------------|
| Weighted Loss | +2-5% | Easy |
| Enhanced Augmentation | +3-7% | Easy |
| Transfer Learning (ResNet) | +5-10% | Medium |
| Attention Mechanism | +2-4% | Medium |
| Label Smoothing | +1-3% | Easy |
| Better Optimizer (AdamW + OneCycle) | +2-5% | Easy |
| **Combined Effect** | **+10-20%** | - |

---

## 🔑 Key Takeaways

1. **Data Quality Matters**: Balance classes and augment appropriately
2. **Modern Architectures Help**: ResNet >> Custom CNN from scratch
3. **Regularization is Critical**: Use multiple techniques (dropout, label smoothing, etc.)
4. **Monitor Everything**: Track metrics, visualize errors, analyze failures
5. **Iterate Quickly**: Start simple, add complexity gradually

---

## 📚 Additional Resources

- **FER Datasets**: FER2013, CK+, AffectNet, RAF-DB
- **Papers**: "Deep Facial Expression Recognition: A Survey" (2020)
- **Libraries**: timm (PyTorch Image Models), albumentations (augmentation)
- **Tools**: Weights & Biases, TensorBoard, Netron (model visualization)

---

**Good luck with your improvements! 🚀**